**Задача**: NER. **Язык**: русский. **Датасет**: https://huggingface.co/datasets/wikiann/viewer/ru

#Установка пакетов и импорт библиотек

In [ ]:
!pip install transformers==4.28.1
!pip install datasets
!pip install seqeval
!pip install -U sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, AutoModelForTokenClassification
from transformers.optimization import AdamW
from collections import Counter
import numpy as np
from torch.utils.data.dataset import Dataset
import scipy
from datasets import load_dataset, load_metric

#Загрузка и подготовка данных

In [ ]:
dataset = load_dataset("wikiann", "ru")

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
ner_tags = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

train_data = [{"words": elem["tokens"], "labels": [ner_tags[index] for index in elem["ner_tags"]]} for elem in dataset["train"]]
dev_data = [{"words": elem["tokens"], "labels": [ner_tags[index] for index in elem["ner_tags"]]} for elem in dataset["validation"]]
test_data = [{"words": elem["tokens"], "labels": [ner_tags[index] for index in elem["ner_tags"]]} for elem in dataset["test"]]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased", use_fast=True, add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def make_subtoken_mask(mask, mode="first", has_cls=True, has_eos=True):
    if has_cls:
        mask = mask[1:]
    if has_eos:
        mask = mask[:-1]
    is_word = list((first != second) for first, second in zip(mask[1:], mask[:-1]))
    is_word = ([True] + is_word) if mode == "first" else (is_word+[True])
    if has_cls:
        is_word = [False] + is_word
    if has_eos:
        is_word.append(False)
    return is_word

In [ ]:
class UDDataset(Dataset):

    def __init__(self, data, tokenizer, min_count=3, tags=None, subtoken_mode="first"):
        self.data = data
        self.tokenizer = tokenizer
        if tags is None:
            tag_counts = Counter([tag for elem in data for tag in elem["labels"]])
            self.tags_ = ["<PAD>", "<UNK>"] + [x for x, count in tag_counts.items() if count >= min_count]
        else:
            self.tags_ = tags
        self.tag_indexes_ = {tag: i for i, tag in enumerate(self.tags_)}
        self.unk_index = 1
        self.ignore_index = -100
        self.subtoken_mode = subtoken_mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        tokenization = self.tokenizer(item["words"], is_split_into_words=True)
        last_subtoken_mask = make_subtoken_mask(tokenization.word_ids(), mode=self.subtoken_mode)
        answer = {"input_ids": tokenization["input_ids"], "mask": last_subtoken_mask}
        if "labels" in item:
            labels = [self.tag_indexes_.get(tag, self.unk_index) for tag in item["labels"]]
            zero_labels = np.array([self.ignore_index] * len(tokenization["input_ids"]), dtype=int)
            zero_labels[last_subtoken_mask] = labels
            answer["labels"] = zero_labels
        return answer

In [ ]:
train_dataset = UDDataset(train_data, tokenizer)

#Обучение

In [ ]:
class NERMetricScorer:

    def __init__(self, names):
        self.names = names
        self.metric = load_metric("seqeval")

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        pred_labels = np.argmax(logits, axis=-1)
        word_labels = [
            [self.names[index] for index in sent_labels if index != -100] for sent_labels in labels
        ]
        pred_word_labels = [
            [self.names[pred_index] for pred_index, index in zip(pred_sent_labels, sent_labels) if index != -100]
            for pred_sent_labels, sent_labels in zip(pred_labels, labels)
        ]
        results = self.metric.compute(references=word_labels, predictions=pred_word_labels)
        answer = dict()
        print(results)
        for key, value in results.items():
            if isinstance(value, dict) and "f1" in value:
                answer[f"{key}_f1"] = value["f1"]
            elif key.startswith("overall_"):
                answer[key[8:].title()] = value
        return answer

model = AutoModelForTokenClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=len(train_dataset.tags_))

val_dataset = UDDataset(dev_data, tokenizer, tags=train_dataset.tags_)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
training_args = TrainingArguments(
    output_dir="trainer_logs", evaluation_strategy="steps", save_strategy='steps',
    logging_strategy="steps", save_total_limit=2,
    num_train_epochs=2, eval_steps=500, disable_tqdm=False,
    metric_for_best_model='F1',
    warmup_ratio=0.1
)
trainer = Trainer(
    model=model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=NERMetricScorer(train_dataset.tags_).compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expec

Step,Training Loss,Validation Loss,Loc F1,Org F1,Per F1,Precision,Recall,F1,Accuracy
500,0.984800,0.275895,0.772463,0.682852,0.872096,0.727886,0.821956,0.772066,0.923064
1000,0.242200,0.194082,0.875635,0.773065,0.939049,0.849590,0.872872,0.861073,0.949217
1500,0.197300,0.190136,0.882341,0.781977,0.947427,0.847697,0.890790,0.868709,0.951735
2000,0.194700,0.167016,0.904110,0.823603,0.956329,0.893341,0.892979,0.893160,0.959476
2500,0.179200,0.158762,0.907922,0.831683,0.957934,0.896134,0.900276,0.898200,0.961752
3000,0.142300,0.162155,0.905335,0.826087,0.958753,0.889528,0.902222,0.895830,0.961368
3500,0.145200,0.165711,0.912135,0.833710,0.960155,0.895475,0.906437,0.900923,0.961567
4000,0.128200,0.160050,0.909468,0.838589,0.958655,0.893048,0.909194,0.901049,0.962891
4500,0.124800,0.155622,0.912223,0.842803,0.958138,0.896648,0.910897,0.903716,0.963673
5000,0.117600,0.157395,0.910547,0.841292,0.957491,0.893929,0.910816,0.902293,0.963047


{'LOC': {'precision': 0.7284514243973703, 'recall': 0.8221352019785655, 'f1': 0.7724632068164214, 'number': 4852}, 'ORG': {'precision': 0.6333479789103691, 'recall': 0.7407502569373073, 'f1': 0.6828517290383704, 'number': 3892}, 'PER': {'precision': 0.8374358974358974, 'recall': 0.9097493036211699, 'f1': 0.8720961281708944, 'number': 3590}, 'overall_precision': 0.7278862722573234, 'overall_recall': 0.8219555699691908, 'overall_f1': 0.7720661031147666, 'overall_accuracy': 0.923063788613953}
{'LOC': {'precision': 0.8813948632282314, 'recall': 0.8699505358615004, 'f1': 0.8756353075407115, 'number': 4852}, 'ORG': {'precision': 0.74712368168744, 'recall': 0.8008735868448099, 'f1': 0.7730654761904762, 'number': 3892}, 'PER': {'precision': 0.9237402317434654, 'recall': 0.954874651810585, 'f1': 0.9390494452814684, 'number': 3590}, 'overall_precision': 0.8495896464646465, 'overall_recall': 0.8728717366628831, 'overall_f1': 0.8610733423978245, 'overall_accuracy': 0.9492166934646196}
{'LOC': {'pr

TrainOutput(global_step=5000, training_loss=0.24562934799194336, metrics={'train_runtime': 953.421, 'train_samples_per_second': 41.954, 'train_steps_per_second': 5.244, 'total_flos': 442068597920160.0, 'train_loss': 0.24562934799194336, 'epoch': 2.0})

Посчитаем метрики на тестововм датасете.

In [ ]:
test_dataset = UDDataset(test_data, tokenizer, tags=train_dataset.tags_)
predictions = trainer.predict(test_dataset)
print(predictions.metrics)

{'LOC': {'precision': 0.8916362072672543, 'recall': 0.9094298245614035, 'f1': 0.9004451199652589, 'number': 4560}, 'ORG': {'precision': 0.8382388713208465, 'recall': 0.8458517427589592, 'f1': 0.842028100183262, 'number': 4074}, 'PER': {'precision': 0.9406896551724138, 'recall': 0.9624611910810048, 'f1': 0.9514508928571428, 'number': 3543}, 'overall_precision': 0.8882699604423993, 'overall_recall': 0.9035887328570256, 'overall_f1': 0.895863865819899, 'overall_accuracy': 0.961746717540119}
{'test_loss': 0.17282509803771973, 'test_LOC_f1': 0.9004451199652589, 'test_ORG_f1': 0.842028100183262, 'test_PER_f1': 0.9514508928571428, 'test_Precision': 0.8882699604423993, 'test_Recall': 0.9035887328570256, 'test_F1': 0.895863865819899, 'test_Accuracy': 0.961746717540119, 'test_runtime': 32.1142, 'test_samples_per_second': 311.389, 'test_steps_per_second': 38.924}


Метрики высокие: Accuracy - 0.96, F1-мера - 0.89. Посмотрим, как модель будет справляться с новыми примерами.

#Проверка на новых примерах и выводы

In [ ]:
import scipy

def predict_with_trainer(trainer, dataset, classes):
    predictions = trainer.predict(dataset)
    answer = []
    for elem, curr_predictions in zip(dataset, predictions.predictions):
        mask = elem["mask"]
        probs = scipy.special.softmax(curr_predictions, axis=-1)[:len(mask)]
        best_indexes = np.argmax(probs, axis=-1)[elem["mask"]]
        best_labels = np.take(classes, best_indexes)
        best_probs = np.max(probs, axis=-1)[elem["mask"]]
        curr_answer = {"labels": best_labels, "probs": best_probs}
        answer.append(curr_answer)
    return answer

In [ ]:
#1
data_sample = {
    "words": "В Пушкине много всего интересного .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

В O 99.75
Пушкине B-LOC 99.78
много O 99.81
всего O 99.70
интересного O 99.76
. O 99.88


Пушкин уверенно распознался как локация - хороший результат.

In [ ]:
#2
data_sample = {
    "words": "Река Лена одна из самых протяженных .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

Река O 89.92
Лена B-LOC 77.08
одна O 99.87
из O 99.88
самых O 99.87
протяженных O 99.81
. O 99.90


Лена распознана как локация тоже с достаточно большой вероятностью.

In [ ]:
#3
data_sample = {
    "words": "Праздник Восьмое марта появился в двадцатом веке .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

Праздник O 81.96
Восьмое B-ORG 57.21
марта I-ORG 86.45
появился O 99.93
в O 99.90
двадцатом B-LOC 47.44
веке I-ORG 70.99
. O 99.94


Тут уже начались проблемы: март и век распознаны как ORG с достаточно большой вероятностью. Стоит однако отметить, что несмотря на то, что модель неверно определила Восьмое как ORG и двадцатом как LOC, вероятности у этих меток невысокие.

In [ ]:
#4
data_sample = {
    "words": "Гостиница 'Украина' одна из самых дорогих не только в России, но и в Европе.".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

Гостиница B-ORG 85.58
'Украина' I-ORG 70.57
одна O 99.94
из O 99.94
самых O 99.93
дорогих O 99.91
не O 99.93
только O 99.91
в O 99.92
России, B-LOC 99.74
но O 99.94
и O 99.92
в O 99.92
Европе. B-LOC 99.60


Гостиница и Украина распознались как организация с достаточно большой вероятностью - хорший результат.

In [ ]:
#5
data_sample = {
    "words": "31 июля 2017 года власти Лос-Анджелеса и руководство МОК заявили, что в 2024 году летние Олимпийские игры пройдут в Париже .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

31 O 96.63
июля O 93.54
2017 O 92.49
года O 99.81
власти O 99.84
Лос-Анджелеса B-LOC 99.51
и O 99.95
руководство O 99.85
МОК B-ORG 96.31
заявили, O 99.94
что O 99.94
в O 99.91
2024 O 78.70
году O 88.28
летние B-ORG 84.02
Олимпийские I-ORG 92.16
игры I-ORG 93.57
пройдут O 99.62
в O 99.93
Париже B-LOC 99.58
. O 99.95


Итак, летние Олимпийские игры были, сюдя по всему, определены как единая группа и распознаны как организация. Вероятность достаточно большая, хотя более точной была бы метка O.

In [ ]:
#6
data_sample = {
    "words": "Илон Маск является CEO Теслы и SpaceX .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

Илон B-PER 99.58
Маск I-PER 99.57
является O 99.96
CEO O 99.93
Теслы B-ORG 93.40
и O 99.95
SpaceX B-ORG 99.64
. O 99.96


В этом примере проблем нет. Даже CEO дана метка O с большой вероятностью, хотя модель могла бы приписать PER или ORG.

In [ ]:
#7
data_sample = {
    "words": "На протяжении многих веков планета Венера вдохновляла ученых .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

На O 99.95
протяжении O 99.94
многих O 99.93
веков O 99.89
планета B-LOC 67.36
Венера I-LOC 76.85
вдохновляла O 99.94
ученых O 99.74
. O 99.96


Планет Венера выделена как группа, и ей приписана метка LOC. Все хорошо.

In [ ]:
#8
data_sample = {
    "words": "4 июля в США празднуется День независмости - день, когда была принята Декларация независимости от Королевства Великобритании. ".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

4 B-LOC 71.47
июля I-LOC 49.85
в O 99.91
США B-LOC 99.51
празднуется O 99.92
День O 68.33
независмости O 97.56
- O 99.89
день, O 99.88
когда O 99.93
была O 99.92
принята O 99.87
Декларация B-ORG 98.16
независимости I-ORG 95.32
от I-ORG 87.71
Королевства I-ORG 86.10
Великобритании. I-ORG 90.72


Тут уже есть проблемы. Во-первых, дата определена как локация, Декларация независимости - как организация, причем с большой вероятностью. И, видимо, "от Королевства Великобритании" распознано как единая группа, потому что каждому слову дана метка ORG тоже с большой вероятностью. Однако, модель справилась с Днем независимости и определила его как O.

In [ ]:
#9
data_sample = {
    "words": "Считается, что 'Евгений Онегин' по праву совершил революцию в русской литературе .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

Считается, O 99.92
что O 99.95
'Евгений O 99.69
Онегин' I-PER 92.91
по O 99.95
праву O 99.94
совершил O 99.94
революцию O 99.87
в O 99.89
русской B-LOC 42.96
литературе O 54.32
. O 99.95


Интерсеный результат. Евгений Онегин не был выделен как единая группа, и модель правильно отнесла Евгения к O, но не справилась с Онегиным. Также есть проблема с прилагательным "русский" - дана метка локации, хотя правильнее было бы дать метку O.

In [ ]:
#10
data_sample = {
    "words": "Комментатор объявил, что первый гол в ЧМ-2002 в Эквадоре был забит Белугой .".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")

Комментатор O 99.85
объявил, O 99.95
что O 99.96
первый O 99.93
гол O 99.92
в O 99.89
ЧМ-2002 B-ORG 92.60
в O 99.79
Эквадоре B-LOC 74.25
был O 99.95
забит O 99.93
Белугой B-PER 94.17
. O 99.96


Результат в целом хороший. Как и ожидалось, у ЧМ-2002 метка ORG, хотя в реальности у него скорее должна быть метка O, Белуге дана метка PER с высокой вероятностью.

In [ ]:
#11
data_sample = {
    "words": "Исследователи полагают, что Серебряный век окончился с началом Гражданской войны или в год смерти Александра Блока, во всяком случае, это было позже распада ЛЕФа.".split()
}
test_dataset = UDDataset([data_sample], tokenizer, tags=train_dataset.tags_)
predictions = predict_with_trainer(trainer, test_dataset, classes=train_dataset.tags_)
for word, label, prob in zip(data_sample["words"], predictions[0]["labels"], predictions[0]["probs"]):
    print(word, label, f"{100*prob:.2f}")



Исследователи O 99.94
полагают, O 99.94
что O 99.95
Серебряный O 51.96
век O 70.18
окончился O 99.93
с O 99.94
началом O 99.88
Гражданской B-ORG 99.12
войны I-ORG 99.06
или O 99.94
в O 99.94
год O 99.92
смерти O 99.91
Александра B-PER 99.58
Блока, I-PER 98.84
во O 99.94
всяком O 99.93
случае, O 99.93
это O 99.93
было O 99.94
позже O 99.93
распада O 99.91
ЛЕФа. B-ORG 98.36


И здесь результат в целом хороший. Модель справилась с "Серебряным веком" и верно отнесла его к O. Хорошо, что "Гражданская война" выделена как группа, но метка ей приписана скорее неправильная, причем с высокой вероятностью. С Алекснадром Блоком и ЛЕФом проблем не возникло.

**Вывод**

В целом, при тестрировании на новых примерах модель показала себя хорошо. Есть проблемы с выделением дат, также иногда вместо метки O приписывается метка ORG, и есть сложности с прилагательными, образованными от названий локаций, что ожидаемо. Но даже с вполне неоднозначными примерами модель справилась хорошо.